In [ ]:
class Chatbot():
    #Will implement later

In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

loader = TextLoader('./poker.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)


embeddings = HuggingFaceEmbeddings()

c:\Users\sathu\anaconda3\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
Created a chunk of size 4257, which is longer than the specified 1000
Created a chunk of size 3099, which is longer than the specified 1000
C:\Users\sathu\AppData\Local\Temp\ipykernel_37548\3970803738.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
C:\Users\sathu\AppData\Local\Temp\ipykernel_37548\3970803738.py:12: LangChainDeprecationW

In [15]:
from langchain.vectorstores import Pinecone
from pinecone import Pinecone as PC
import os
# Initialize Pinecone client

PC(api_key= os.getenv('PINECONE_API_KEY'))

# Define Index Name
index_name = "langchain-demo"

# Checking Index
if index_name not in PC.list_indexes():
  # Create new Index
  PC.create_index(name=index_name, metric="cosine", dimension=768)
  docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)
else:
  # Link to the existing index
  docsearch = Pinecone.from_existing_index(index_name, embeddings)

TypeError: list_indexes() missing 1 required positional argument: 'self'

In [ ]:
from langchain.llms import HuggingFaceHub

# Define the repo ID and connect to Mixtral model on Huggingface
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(
  repo_id=repo_id, 
  model_kwargs={"temperature": 0.8, "top_k": 50}, 
  huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
)